# PROJECT_2. Подгрузка новых данных. Уточнение анализа

## 1. Знакомство с датасетом

CANDIDATE
Таблица хранит в себе общие данные по кандидатам: id, пол, возраст, желаемая должность, город, вид занятости, текущая должность, дата обновления записи и зарплата.

CITY
city — таблица-справочник для наших кандидатов — хранит код города и его название.

CANDIDATE_TIMETABLE_TYPE
Это дополнительная таблица. Она существует для организации связи многие-ко-многим, так как есть много кандидатов и у них может быть несколько подходящих типов рабочего графика.

TIMETABLE_TYPE
Это таблица-справочник вариантов рабочего графика, подходящего кандидату.

## 2. Предварительный анализ данных

Задание 2.1
Рассчитайте максимальный возраст (max_age) кандидата в таблице.

Код:
SELECT MAX(age) /* максимальный возраст */
  FROM hh.candidate /* из таблицы candidate в папке hh */

Решение: 100
Вывод: это значение явно не имеет отношения к реальности.

Задание 2.2
Теперь давайте рассчитаем минимальный возраст (min_age) кандидата в таблице.

Код:
SELECT MIN(age) /* минимальный возраст */
  FROM hh.candidate /* из таблицы candidate в папке hh */

Решение: 14
Вывод: значение вполне может быть настоящим

Задание 2.3
Попробуем «почистить» данные. Напишите запрос, который позволит посчитать для каждого возраста (age) сколько (cnt) человек этого возраста у нас есть.
Отсортируйте результат по возрасту в обратном порядке.

Код:
SELECT 
    age AS age, /* возраст кандидата в колонку age */
    COUNT(age) AS cnt /* подсчет количества кандидатов текущего возраста */ 
 FROM hh.candidate
GROUP BY age /* групптровка по возрасту */
ORDER BY 1 DESC /* сортировка по возрасту в порядке убывания */

Решение: файл '2.3.png' в папке 'screens'
Вывод: самый распространенный возраст соискателей - от 22 до 38 лет. Больше всего кандидатов в возрасте 30 лет. Такое распределение вполне логично, поскольку к 22 годам студенты заканчивают обучение и начинают поиск работы. К 30 годам люди уже обзаводятся семьями, переосмысливают жизнь, часто  хотят сменить место работы.

Задание 2.4
По данным Росстата, средний возраст занятых в экономике России составляет 39.7 лет. Мы округлим это значение до 40. Найдите количество кандидатов, которые старше данного возраста. Не забудьте отфильтровать «ошибочный» возраст 100.

Код:
SELECT COUNT(*) /* подсчет количества соискателей */
  FROM hh.candidate
 WHERE age > 40 /* фильтр - возраст больше 40 */
   AND age <> 100 /* 100-летнего не считаем */

Решение: 6263
Вывод: Всего в датасете представлены резюме 44744 соискателей, старше 40 лет - 6263 человека, это около 14%. Данные Росстата подтверждаются. Скорее всего, это связано с тем, что в этом возрасте люди дорожат своей работой, не стремятся ее сменить, а при поиске реже используют ресурсы типа HH.ru.

## 3. Глобальный анализ показателей

Задание 3.1
Для начала напишите запрос, который позволит узнать, сколько (cnt) у нас кандидатов из каждого города (city).
Формат выборки: city, cnt.
Группировку таблицы необходимо провести по столбцу title, результат отсортируйте по количеству в обратном порядке.

Код:
SELECT 
    city.title AS city, /* название города в колонку city */
    COUNT(*) AS cnt /* подсчет кондидатов в этом городе */
 FROM hh.city city /* из таблицы city */
 JOIN hh.candidate cand ON city.id = cand.city_id /* соудиненной с таблицей candidate по id города */
GROUP BY city.title /* группировка по названию города */
ORDER BY 2 DESC /* соортировка по количеству кандидатов в порядке убывания */

Решение: файл '3.1.png' в папке 'screens'
Вывод: лидер с большим отрывом по количеству соискателей - Москва, затем идут Санкт-Петербург (в 4 раза меньше), Краснодар, Новосибирск. Это города-милионники, соответственно, и ищущих работу там больше.

Задание 3.2
Москва бросается в глаза как, пожалуй, самый активный рынок труда. Напишите запрос, который позволит понять, каких кандидатов из Москвы устроит «проектная работа».
Формат выборки: gender, age, desirable_occupation, city, employment_type.
Отсортируйте результат по id кандидата.

Код:
SELECT
    cand.gender AS gender, /* первая колонка - пол */
    cand.age AS age, /* возраст */
    cand.desirable_occupation AS desirable_occupation, /* желаемая должность */
    city.title AS city, /* название города */
    cand.employment_type AS employment_type /* вид занятости */
FROM hh.candidate cand /* из таблицы candidate */
JOIN hh.city city ON cand.city_id = city.id /* соединенной с таблицей city по id города */ 
WHERE city.title = 'Москва' /* условие 1 - кандидат из Москвы */
      AND cand.employment_type LIKE '%проектная работа%' /* условие 2 - один из видов занятости - проектная работа */
ORDER BY cand.id /* сортировка по id кандидатов в порядке возрастания */

Решение: файл '3.2.png' в папке 'screens'
Вывод: выборка получилась большая, в Москве многие готовы работать "на проектах". 

Задание 3.3
Данных оказалось многовато. Отфильтруйте только самые популярные IT-профессии — разработчик, аналитик, программист.
Обратите внимание, что данные названия могут быть написаны как с большой, так и с маленькой буквы.
Отсортируйте результат по id кандидата.

Код:
SELECT
    cand.gender AS gender, /* первая колонка - пол */
    cand.age AS age, /* возраст */
    cand.desirable_occupation AS desirable_occupation, /* желаемая должность */
    city.title AS city, /* город */
    cand.employment_type AS employment_type /* вид занятости */
 FROM hh.candidate cand /* из таблицы candidate в папке hh */
 JOIN hh.city city ON cand.city_id = city.id /* соединенной с таблицей city по id города */
WHERE city.title = 'Москва' /* условие 1: кандидат из Москвы, */
      AND cand.employment_type LIKE '%проектная работа%' /* условие 2: один из видов занятости - проектная работа */
      AND (lower(cand.desirable_occupation) LIKE '%разработчик%'
       OR lower(cand.desirable_occupation) LIKE '%аналитик%'
       OR lower(cand.desirable_occupation) LIKE '%программист%')
ORDER BY cand.id /* сортируем по id кандидатов в порядке возрастания */

Решение: файл '3.3.png' в папке 'screens'
Вывод: выборка получилась довольно большая, т.к. "разработчик", "аналитик", "программист" - довольно расплывчатые определения.

Задание 3.4
Для общей информации попробуйте выбрать номера и города кандидатов, у которых занимаемая должность совпадает с желаемой.
Формат выборки: id, city.
Отсортируйте результат по городу и id кандидата.

Код:
SELECT cand.id AS id, /* колонка id кандидата */
       city.title AS city /* название города */
 FROM hh.candidate cand /* из таблицы candidate */
 JOIN hh.city city ON cand.city_id = city.id /* соединенной с таблицей city по id города */
WHERE cand.desirable_occupation = cand.current_occupation /* фильтр: желаемая должность = занимаемая должность */
ORDER BY 2, 1 /* сортировка по городу, затем по id кандидата */

Решение: файл '3.4.png' в папке 'screens'
Вывод: выборка получилась небольшая, значит, некоторые соискатели хотят поменять только место работы.

Задание 3.5
Определите количество кандидатов пенсионного возраста.
Пенсионный возраст для мужчин наступает в 65 лет, для женщин — в 60 лет.

Код:
SELECT COUNT(*) |* считаем всех кандидатов */
  FROM hh.candidate
 WHERE age <> 100 /* кроме 100-летнего */
    AND ((gender = 'M' AND age >= 65) /* мужчин старше или равно 65 */
        OR (gender = 'F' AND age >= 60)) /* женщин старше или равно 60 */

Решение: 75
Вывод: кандидатов пенсионного возраста очень мало. Вряд ли пенсионеры у нас не работают, скорее они не пользуются HH.ru для поиска работы.


## 4. Анализ кандидатов для заказчиков

Задание 4.1
Для добывающей компании нам необходимо подобрать кандидатов из Новосибирска, Омска, Томска и Тюмени, которые готовы работать вахтовым методом.
Формат выборки: gender, age, desirable_occupation, city, employment_type, timetable_type.
Отсортируйте результат по городу и номеру кандидата.

Код:
SELECT
    cand.gender AS gender, /* столбец gender - пол */
    cand.age AS age, /* возраст */
    cand.desirable_occupation AS desirable_occupation, /* желаемая должность */
    city.title AS city, /* название города */
    cand.employment_type AS employment_type, /* вид занятости */
    tt.title AS timetable_type /* тип рабочего графика */
FROM hh.candidate cand /* из таблицы candidate */
JOIN hh.city city /* соединенной с таблицей city */
  ON cand.city_id = city.id /* по id города */
JOIN hh.candidate_timetable_type ctt /* присоединяем таблицу candidate_timetable_type */
  ON cand.id = ctt.candidate_id /* ключ - id кандидата */
JOIN hh.timetable_type tt /* присоединяем timetable_type */
  ON ctt.timetable_id = tt.id /* ключ - id рабочего графика */
WHERE (city.title IN ('Новосибирск', 'Омск', 'Томск', 'Тюмень')) /* условие 1 - город из перечисленных */
  AND (tt.title = 'вахтовый метод') /* условие 2 - у соискателя указан вахтовый метод */
ORDER BY 4, cand.id /* сортировка по городу и id кандидата */

Решение: файл '4.1.png' в папке 'screens'
Вывод: даже при таких жестких условиях-фильтрах (небольшие города, непопулярный график работы) удалось собрать неплохую выборку кандидатов.

Задание 4.2
Для заказчиков из Санкт-Петербурга нам необходимо собрать список из 10 желаемых профессий кандидатов из того же города от 16 до 21 года (в выборку включается 16 и 21, сортировка производится по возрасту) с указанием их возраста, а также добавить строку Total с общим количеством таких кандидатов. Напишите запрос, который позволит получить выборку вида:

Код:
(SELECT 
    cand.desirable_occupation AS desirable_occupation, -- желаемая должность 
    cand.age AS age -- возраст 
   FROM hh.candidate cand -- из таблицы candidate
   JOIN hh.city ct -- соединенной с таблицей city
     ON cand.city_id = ct.id -- по id города
  WHERE ct.title = 'Санкт-Петербург' -- условие 1: город - Санкт-Петербург
    AND cand.age BETWEEN 16 AND 21 -- условие 2: кандидаты в возрасте от 16 до 21
  ORDER BY cand.age -- сортировка по возрасту кандидатов
  LIMIT 10) -- вывод только первых 10 строк

UNION ALL -- объединяем оба запроса

(SELECT 
   'Total', -- в первом столбце слово 'Total'
   COUNT(*) -- во втором количество кандидатов, соответствующих условиям
 FROM hh.candidate cand -- из таблицы candidate
 JOIN hh.city ct -- соединенной с таблицей city
   ON cand.city_id = ct.id -- по id города
WHERE ct.title = 'Санкт-Петербург' -- условие 1: город - Санкт-Петербург
  AND cand.age BETWEEN 16 AND 21) -- условие 2: кандидаты в возрасте от 16 до 21

Решение: файл '4.2.png' в папке 'screens'
Вывод: мы можем выставлять какие угодно условия согласно требованиям заказчика, создавать выборки с промежуточными значениями.

# Общий вывод:
В датасете представлены резюме 44744 кандидатов.
Максимальный возраст - 100 лет, это смело можно считать выбросом. Минимальный возраст соискателя - 14 лет. Основная масса кандидатов - в возрасте от 22 до 38 лет. Максимальное количество соискателей - в возрасте 30 лет. Такое распределение вполне логично, поскольку к 22 годам студенты заканчивают обучение и начинают поиск работы. К 30 годам люди уже обзаводятся семьями, переосмысливают жизнь, часто  хотят сменить место работы. Кандидатов пенсионного возраста очень мало. Вряд ли пенсионеры у нас не работают, скорее они не пользуются HH.ru для поиска работы.
Некоторые соискатели хотят поменять только место работы с сохранением занимаемой должности.
Лидер среди городов по количеству соискателей - Москва (больше 16 тыс.чел.), затем идут Санкт-Петербург (в 4 раза меньше), Краснодар, Новосибирск. Это города-милионники, соответственно, и ищущих работу там больше.
При поиске сотрудников по названию самых популярных IT-профессий — разработчик, аналитик, программист - нужно учитывать не только их написание, но и то, что эти опредеkения очень расплывчаты.
Даже при наличии ряда жестких условий (не самый распространенный возраст, маленькие северные города, непопулярные графики работы) можно сделать хорошую выборку подходящих кандидатов.